# **EMOTION DETECTION**

In [1]:
# Getting all dependencies

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import pickle


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Setting up the directories

TRAINING_DIR = './images/train'
TEST_DIR = './images/validation'

In [5]:
# Making dataframes for the training and test data (with images and labels)

def create_dataframe(directory):
    image_paths = []
    labels = []

    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):
            for image_name in os.listdir(label_dir):
                image_path = os.path.join(label_dir, image_name)
                image_paths.append(image_path)
                labels.append(label)
            print(f"Completed {label_dir}")
    return image_paths, labels

In [ ]:
# Making Train and Test dataframes

train_df = pd.DataFrame()
train_df['image_path'], train_df['label'] = create_dataframe(TRAINING_DIR)

test_df = pd.DataFrame()
test_df['image_path'], test_df['label'] = create_dataframe(TEST_DIR)
print("Train and test dataframes created successfully.")

print(train_df)
print(test_df)

Completed ./images/train\angry
Completed ./images/train\disgust
Completed ./images/train\fear
Completed ./images/train\happy
Completed ./images/train\neutral
Completed ./images/train\sad
Completed ./images/train\surprise
Completed ./images/validation\angry
Completed ./images/validation\disgust
Completed ./images/validation\fear
Completed ./images/validation\happy
Completed ./images/validation\neutral
Completed ./images/validation\sad
Completed ./images/validation\surprise
Train and test dataframes created successfully.
                             image_path     label
0            ./images/train\angry\0.jpg     angry
1            ./images/train\angry\1.jpg     angry
2           ./images/train\angry\10.jpg     angry
3        ./images/train\angry\10002.jpg     angry
4        ./images/train\angry\10016.jpg     angry
...                                 ...       ...
28816  ./images/train\surprise\9969.jpg  surprise
28817  ./images/train\surprise\9985.jpg  surprise
28818  ./images/train\sur

In [22]:
# Label Encoding

label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])
y_train = label_encoder.transform(train_df['label'])
y_test = label_encoder.transform(test_df['label'])

In [ ]:
# Image transforms

transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
])

In [32]:
# Custom Dataset class

class EmotionDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('L')
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

In [33]:
# Create datasets and dataloaders

train_dataset = EmotionDataset(train_df['image_path'].tolist(), y_train, transform=transform)
test_dataset = EmotionDataset(test_df['image_path'].tolist(), y_test, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
# Convolutional Neural Network (CNN) model
# The model is a sequential model with convolutional layers, max pooling layers, and dropout layers
# The model is designed to classify images into 7 different classes (emotions)

class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.3),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.3),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.3),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.3)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 3 * 3, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(64, 7)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [35]:
# Configuring the model

model = EmotionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training Model

def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=150):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device).long()

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")

        # Validation step
        
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device).long()
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = 100 * val_correct / val_total
        print(f"Validation Accuracy: {val_accuracy:.2f}%")

train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=150)

100%|██████████| 226/226 [00:15<00:00, 14.88it/s]


Epoch [1/150], Loss: 1.6285, Accuracy: 36.02%
Validation Accuracy: 40.50%


100%|██████████| 226/226 [00:15<00:00, 14.58it/s]


Epoch [2/150], Loss: 1.4783, Accuracy: 43.28%
Validation Accuracy: 47.88%


100%|██████████| 226/226 [00:17<00:00, 12.98it/s]


...

100%|██████████| 226/226 [00:13<00:00, 17.00it/s]


Epoch [87/150], Loss: 0.5451, Accuracy: 80.31%
Validation Accuracy: 65.74%


100%|██████████| 226/226 [00:13<00:00, 16.69it/s]


Epoch [88/150], Loss: 0.5478, Accuracy: 80.26%
Validation Accuracy: 65.53%


  3%|▎         | 6/226 [00:00<00:14, 14.67it/s]

In [ ]:
# Save model

torch.save(model.state_dict(), "emotionDetector_1.pth")
print("Model saved successfully.")

Model saved successfully.
